# Review: PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation

## Paper Reviews

### Prior Reseach

**Point Cloud**는 본질적으로 irregular format의 3D data structure이기 때문에 이를 render한 mesh, voxel 형태로 사용되어 왔음  
→ 그러나 이는 continuous한 대상의 sample이라 할 수 있는 Point Clound를 통해서 quantization하는 접근이기 때문에 artifacts를 낳게 됨  
  
∴ Point Cloud를 통해서 3D representation을 직접 catch하는 Architecture를 통해 mesh, voxel에서 얻지 못하는 강점을 얻을 수 있음  

### Requirements for Architecture Using Point Cloud

Point Cloud는 3D object에 대한 sample point $ (x, y, z) $에 추가적인 정보가 결합된 **unordered** set이라고 할 수 있음  
근본적으로 $ \mathbb{R} $을 넘어선 dimension에 대한 canonical order가 존재하지 않으므로 2D image와 같이 index가 주어진 input으로 다룰 수 없게 만듦  
따라서 Point Cloud를 통해서 3D representation을 찾아내는 network는 다음의 property를 만족해야 함
+ Permutation invariant - input으로 주어지는 point들의 순서가 변화하더라도 같은 object를 나타내는 것을 이해할 수 있어야 함
+ Rigid Transformation invariant - Rigid Transform(e.g translation, rotation, reflection)이 적용되어도 원본 3D object와 동일한 semantic을 파악할 수 있어야 함

→ 이러한 property를 만족할 수 있는 **PointNet Architecture**를 제안

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/192131135-ff74796a-bdb0-4223-9ad9-cfd90ceb66ef.png" alt="1" width="600px" />
</p>

### Symmetric Function for Permutation Invariant

unordered set의 특징을 지닌 Point Cloud를 input으로 활용할 수 있도록 다음과 같은 approach들을 떠올릴 수 있음
+ input을 사용하기 전에 canonical sorting 후 사용
+ 모든 permutation에 대해서 training되는 RNN structure를 baseline으로 사용
+ symmetric function을 통해서 representation을 catch

→ canonical ordering은 $ \mathbb{R}^k $일 경우 $ k $가 커질 수록 point perturbation에 unstable해지는 문제가 있음  
→ RNN structure는 permutation에 어느정도 robust 할 수 있지만 수천 point의 sequence를 다루게 되는 Point Cloud에서 사용하기 어려움

∴ Network가 symmetric function의 특징을 갖게 하도록 Architecture를 유도하는 것이 가장 합리적임

이러한 observation에서 transformed input sequence $ \{ x_1, x_2, \cdots, x_n \} $에 대하여 3D point set representation function $ f $를 다음과 같도록 approximate 할 수 있음

$$ f(\{ x_1, x_2, \cdots, x_n \}) \approx g(h(x_1), \cdots, h(x_n)) \text{    where   } f : 2^{\mathbb{R}^N} \rightarrow \mathbb{R} \text{, } h : \mathbb{R}^N \rightarrow \mathbb{R}^K \text{, } g: \underbrace{\mathbb{R}^K \times \cdots \times \mathbb{R}^K}_{n} \rightarrow \mathbb{R} \text{ symmetric function} $$

+ 각 point를 higher dimension feature로 mapping하는 $ h $는 **각 point sample마다 동일하게 적용되는 MLP**로 구성, 여러개의 MLP를 통해서 다양한 $ f $를 만들어낼 수 있음  
+ 최종적으로 semantic을 정리하기 위한 $ g $ 는 $ K $개의 feature마다 permutation invariant한 성질을 갖는 **(column-wise) max polling**과 single variable function으로 구성   
+ 이 때 $ h $가 mapping하는 codomain dimension $ K $를 **Bottleneck dimension**이라 칭하며, 이는 pont set에서 object의 semantic을 파악하기 위한 **critical set**의 크기를 결정하게 됨

이렇게 approximate하는 것은 실제로 theoretical하게 임의의 continuous set function에 대한 approximation이 될 수 있다는 것을 다음의 Theorem을 통해서 확인할 수 있음  
→ continuous set function에 대한 universal approximation이 가능하다는 것은 학습된 Network에 **perturbed point set이 들어와도 original과 function value의 큰 차이가 없음**을 암시함

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/192131136-f6f9b77b-9206-4dc3-9394-8f1896ac6995.png" alt="2" width="300px" />
</p>

max pooling의 대상이 되는 $ K $개의 feature에서 $ K $의 크기는 semantic을 결정 짓기 위해 활용할 **key points number의 upper bound**를 정하게 됨  
이러한 key points의 집합인 **critical set**을 제외한 나머지는 semantic을 결정 짓기 위해 불필요한 point이며, 이를 제외해도 object의 semantic을 파악할 수 있다는 것   
따라서 $ K $의 크기는 일종의 **bottleneck dimension**으로 성립하게 되며, 이는 Network의 성능에 영향을 미칠 수 있는 hyperparameter라고 할 수 있음

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/192131137-a7058a4f-702c-477f-9076-7485002c8099.png" alt="3" width="300px" />
</p>

실제로 Network가 이렇게 중요한 semantic을 지닌 critical set을 찾아내는 것인지 visualize 해보면 어느 정도 중요한 representation을 catch하고 summarize하고 있음을 알 수 있음

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/192131138-61bb6654-8ab7-435a-a88c-be06c891f183.png" alt="4" width="300px" />
</p>

$ K $의 크기 변화 및 실제 input point의 수에 따른 성능의 변화로 empirical하게 critical set의 중요성을 파악할 수 있음  
$ K $가 증가함에 따라서 그 증가폭이 감소하기는 하지만 performance의 증가를 확인할 수 있음  
또한 절대적으로 input points가 1K 근방을 넘어갈 때 performance가 최대화된다는 점에서 semantic을 파악하기 위한 critical set의 유의미한 lower bound가 존재한다는 것을 암시함  

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/192131139-735629c3-90af-48e3-ad2f-a29ecdf48db5.png" alt="5" width="400px" />
</p>

### Joint Alignment Network

input으로 들어오는 point set이 original에서 어떠한 Rigid Transform이 적용되었다고 해도, original set이 갖는 semantic을 찾아낼 수 있어야 함  
이를 대처하기 위한 방법 중 하나로 Network는 original points에 대해서 어떠한 transform이 가해져 input points가 되었는지 **transform을 예측**할 수 있도록 구성함  
따라서 2D image에서 적용된 transform matrix을 예측하고 이를 복구하는 **Spatial Transformer**의 구조를 차용한 subnetwork **T-net**을 추가

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/192131140-0c909167-472a-4eac-a9df-0293d6bb04b2.png" alt="6" width="500px" />
</p>



저자들은 단순히 input points만이 아니라 mapping된 feature space에서의 point에서도 transform에 robust 할 수 있도록 feature space 상의 transform도 예측할 수 있도록 추가함  
그러나 $ (x, y, z) $의 $\mathbb{R}^3$ space가 아닌 더 높은 $\mathbb{R}^K$ space에서의 spatial transform을 예측하는 것은 optimization에서 어려움을 만듦   
따라서 추가로 T-net이 예측하는 transform이 **orthogonal transform**이 되도록 loss에 반영하여 input semantic이 손상되지 않는 orthogonal matrix만을 예측하도록 유도함  

$$ L_{reg} = \|| I - AA^T \||^2_F$$
  

### Local and Global Information Aggregation

Point Cloud를 통해 classification을 수행할 때와 달리 segmentation은 하나의 point를 볼 때 **global context**를 동시에 볼 수 있어야 함  
그러나 지금까지 사용한 $ f $의 approximation으로 반환되는 vector는 단순한 input points들의 summarization으로 생성되는 **global signature**임    
따라서 각 points를 실제로 segmentation하기 위해서는 각 points의 representation을 catch하는 과정에서 **해당 정보가 동시에 반영되어야 함**  

→ max polling 후 얻어지는 global context를 **각 point의 representation과 하나씩 모두 concatenate**하여 이를 통해 segmentation을 수행

## Implementation Reviews

T-net/regularizer과 PointNet feature extractor implementation  
→ point-wise operation을 **1 size 1-D convolution으로 구현**했음


In [ ]:
class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x
    
def feature_transform_regularizer(trans):
    d = trans.size()[1]
    batchsize = trans.size()[0]
    I = torch.eye(d)[None, :, :]
    if trans.is_cuda:
        I = I.cuda()
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2,1)) - I, dim=(1,2)))
    return 

In [ ]:
class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True, feature_transform = False):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat

## Reference

https://arxiv.org/abs/1612.00593  
https://arxiv.org/abs/1506.02025  
https://github.com/fxia22/pointnet.pytorch/blob/master/pointnet/model.py
